In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils import data

In [4]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [73]:
EPOCHS = 30
BATCH_SIZE = 64

In [26]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [27]:
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    transform = transform,
    download=True
)

In [28]:
testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    transform = transform,
    download=True
)

In [33]:
train_loader = data.DataLoader(
    dataset = trainset,
    batch_size=BATCH_SIZE)
test_loader = data.DataLoader(
    dataset = testset,
    batch_size= BATCH_SIZE
)

In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128,10)
        
    def forward(self, input):
        flatten = input.view(-1, 784)
        fc1 = F.relu(self.fc1(flatten))
        fc2 = F.relu(self.fc2(fc1))
        output = self.fc3(fc2)
        return output

In [20]:
model = Net().to(DEVICE)

In [22]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [37]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
    
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [71]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [62]:
test   = torch.Tensor([[0,0,1,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0],[1,0,0,0,0,0,0,0,0]])

In [67]:
test.max(1)

torch.return_types.max(
values=tensor([1., 1., 1.]),
indices=tensor([2, 4, 0]))

In [70]:
test.max(1, keepdim=True)[1].shape

torch.Size([3, 1])

In [74]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'{epoch} Test Loss : {test_loss}, accuracy : {test_accuracy}')

1 Test Loss : 0.3703068133354187, accuracy : 86.72
2 Test Loss : 0.3683851508617401, accuracy : 86.79
3 Test Loss : 0.3659388162612915, accuracy : 86.84
4 Test Loss : 0.3641966445684433, accuracy : 87.07
5 Test Loss : 0.36273867089748385, accuracy : 87.09
6 Test Loss : 0.36082945609092715, accuracy : 87.2
7 Test Loss : 0.35839357476234435, accuracy : 87.25
8 Test Loss : 0.3573462371110916, accuracy : 87.26
9 Test Loss : 0.35443802402019503, accuracy : 87.38
10 Test Loss : 0.35317495274543764, accuracy : 87.47
11 Test Loss : 0.35192358584403993, accuracy : 87.52
12 Test Loss : 0.35108426275253296, accuracy : 87.48
13 Test Loss : 0.35136865220069885, accuracy : 87.45
14 Test Loss : 0.35019447367191314, accuracy : 87.5
15 Test Loss : 0.3487207622051239, accuracy : 87.51
16 Test Loss : 0.34728842041492464, accuracy : 87.62
17 Test Loss : 0.346106406378746, accuracy : 87.66
18 Test Loss : 0.3445451088666916, accuracy : 87.68
19 Test Loss : 0.3445059564828873, accuracy : 87.73
20 Test Loss :